In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Concatenate, Input, Conv3D, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
import os
from keras.regularizers import l2
from keras.constraints import MinMaxNorm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras import backend as K
# Set default floating-point type to float64
tf.keras.backend.set_floatx('float64')

In [10]:
sorted_concatenated_csv = "C:/Users/Ioannis/Documents/UvA thesis/UvA-thesis/data/test_data_concat/1st run concat.csv"
data = pd.read_csv(sorted_concatenated_csv)
data.drop(columns=['zCOM'], inplace=True)
print(data.head())
data['time'] = (data['mcsteps'] / 10000).astype(int)
data = data[['time'] + [col for col in data.columns if col != 'time']]
data.drop(columns=['mcsteps'], inplace=True)
print(data)
cytokine_columns = ['il8', 'il1', 'il6', 'il10', 'tnf', 'tgf']
smallest_values = data[cytokine_columns].min()
largest_values = data[cytokine_columns].max()

print("Smallest values for each cytokine:")
print(smallest_values)
print("\nLargest values for each cytokine:")
print(largest_values)
def replace_negative_with_zero(data):
    num_negative_values = (data < 0).sum().sum()
    data[data < 0] = 0

    return num_negative_values

cytokine_columns = ['il8', 'il1', 'il6', 'il10', 'tnf', 'tgf']

for col in cytokine_columns:
    num_negatives = replace_negative_with_zero(data[col])
    print(f"Number of negative values replaced with 0 in '{col}': {num_negatives}")

   mcsteps  xCOM  yCOM           il8  il1  il6  il10  tnf  tgf
0        0    33    22  3.750408e-07  0.0  0.0   0.0  0.0  0.0
1        0    11    32  4.793328e-09  0.0  0.0   0.0  0.0  0.0
2        0    30    35  8.997935e-09  0.0  0.0   0.0  0.0  0.0
3        0    17    40  1.781103e-08  0.0  0.0   0.0  0.0  0.0
4        0    21    23  4.789621e-09  0.0  0.0   0.0  0.0  0.0
       time  xCOM  yCOM           il8           il1           il6  \
0         0    33    22  3.750408e-07  0.000000e+00  0.000000e+00   
1         0    11    32  4.793328e-09  0.000000e+00  0.000000e+00   
2         0    30    35  8.997935e-09  0.000000e+00  0.000000e+00   
3         0    17    40  1.781103e-08  0.000000e+00  0.000000e+00   
4         0    21    23  4.789621e-09  0.000000e+00  0.000000e+00   
...     ...   ...   ...           ...           ...           ...   
86155   100    49    35 -8.886183e-11  6.343394e-10  1.206925e-08   
86156   100    36     4  1.334245e-07  1.665160e-07  1.305691e-08   
8

C:\Users\Ioannis\AppData\Local\Temp\ipykernel_21960\3945162383.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data < 0] = 0
C:\Users\Ioannis\AppData\Local\Temp\ipykernel_21960\3945162383.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data < 0] = 0
C:\Users\Ioannis\AppData\Local\Temp\ipykernel_21960\3945162383.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data < 0] = 0
C:\Users\Ioannis\AppData\Local\Tem

In [15]:
# Extract relevant columns
time = data['time'].values.reshape(-1, 1)
xCOM = data['xCOM'].values.reshape(-1, 1)
yCOM = data['yCOM'].values.reshape(-1, 1)
cytokines = data[['il8', 'il1', 'il6', 'il10', 'tnf', 'tgf']].values
# Concatenate inputs
inputs = np.concatenate([time, xCOM, yCOM], axis=1)

# Convert to TensorFlow tensors
inputs_tf = tf.convert_to_tensor(inputs, dtype=tf.float64)
cytokines_tf = tf.convert_to_tensor(cytokines, dtype=tf.float64)

# Check shapes
print("Input shape:", inputs_tf.shape)  # Should be (number_of_samples, 3)
print("Output shape:", cytokines_tf.shape)  # Should be (number_of_samples, 6)

Input shape: (86160, 3)
Output shape: (86160, 6)


In [12]:
# Metrics and lr schedule callback
def lr_schedule(epoch, lr):
    if epoch < 100:
        return 1e-4
    if epoch < 400:
        return 1e-5
    else:
        return 1e-6

def r_squared(y_true, y_pred):
    SS_res = tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))
    SS_tot = tf.keras.backend.sum(tf.keras.backend.square(y_true - tf.keras.backend.mean(y_true)))
    return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())

def average_relative_rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square((y_pred - y_true) / tf.keras.backend.clip(tf.keras.backend.abs(y_true), tf.keras.backend.epsilon(), None))))

def accuracy(y_true, y_pred):
    abs_diff = K.abs(y_true - y_pred)
    threshold = 0.3 * y_true
    accurate_predictions = K.less_equal(abs_diff, threshold)
    accuracy = K.mean(accurate_predictions)
    return accuracy

In [24]:
# Define the PINN model
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden1 = tf.keras.layers.Dense(20, activation='tanh')
        self.hidden2 = tf.keras.layers.Dense(20, activation='tanh')
        self.hidden3 = tf.keras.layers.Dense(20, activation='tanh')
        self.output_layer = tf.keras.layers.Dense(6, activation=None)  # Adjusted for 6 outputs

    def call(self, x):
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        return self.output_layer(x)

# Initialize the model
model = PINN()

# Example parameters for IL-8
D_IL8 = 2.09e-6
mu_IL8 = 0.2
K_EC_IL8 = 234
theta_NA_IL8 = 3.024
params_IL8 = (D_IL8, mu_IL8, K_EC_IL8, theta_NA_IL8)

def compute_pde_residuals(model, x, t, params, cytokine_index):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        
        inputs = tf.concat([x, t], axis=1)
        f = model(inputs)[:, cytokine_index]  # Select the specific cytokine output
        
        # Compute first-order derivatives
        f_t = tape.gradient(f, t)
        
        # Compute second-order derivatives
        f_x = tape.gradient(f, x)
        f_xx = tape.gradient(f_x, x)
        
    D, mu, K_secretion, theta_endocytosis = params
    
    # Ensure parameters are scalars and broadcastable
    D = tf.convert_to_tensor(D, dtype=tf.float64)
    mu = tf.convert_to_tensor(mu, dtype=tf.float64)
    K_secretion = tf.convert_to_tensor(K_secretion, dtype=tf.float64)
    theta_endocytosis = tf.convert_to_tensor(theta_endocytosis, dtype=tf.float64)

    # Correcting the shapes for broadcasting
    f_t = tf.reshape(f_t, (-1,))
    f_xx = tf.reduce_sum(f_xx, axis=1)
    f = tf.reshape(f, (-1,))

    print(f"f_t shape: {f_t.shape}, D shape: {D.shape}, f_xx shape: {f_xx.shape}, mu shape: {mu.shape}, f shape: {f.shape}, K_secretion shape: {K_secretion.shape}, theta_endocytosis shape: {theta_endocytosis.shape}")

    if cytokine_index == 0:  # IL-8
        residual = f_t - D * f_xx + mu * f - K_secretion + theta_endocytosis
    elif cytokine_index == 1:  # IL-1b
        residual = f_t - D * f_xx + mu * f - K_secretion
    # Add other cytokine types similarly

    return residual

def compute_loss(model, x, t, true_values, params, cytokine_index):
    pde_residuals = compute_pde_residuals(model, x, t, params, cytokine_index)
    pde_loss = tf.reduce_mean(tf.square(pde_residuals))
    
    data_loss = tf.reduce_mean(tf.square(model(tf.concat([x, t], axis=1))[:, cytokine_index] - true_values[:, cytokine_index]))
    
    total_loss = pde_loss + data_loss
    return total_loss

def train(model, x, t, true_values, params, cytokine_index, epochs, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = compute_loss(model, x, t, true_values, params, cytokine_index)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Initialize and train the model for IL-8 (cytokine_index = 0)
train(model, inputs_tf, inputs_tf[:, 0:1], cytokines_tf, params_IL8, cytokine_index=0, epochs=1000, learning_rate=1e-3)

# Example evaluation
x_test = np.random.rand(100, 1)
t_test = np.random.rand(100, 1)
xCOM_test = np.random.rand(100, 1)
yCOM_test = np.random.rand(100, 1)

# Ensure the test input includes time, xCOM, and yCOM
inputs_test = np.concatenate([t_test, xCOM_test, yCOM_test], axis=1)
inputs_test_tf = tf.convert_to_tensor(inputs_test, dtype=tf.float64)

predicted_values = model(inputs_test_tf)

f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
Epoch 0, Loss: 53377.083285396846
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocytosis shape: ()
f_t shape: (86160,), D shape: (), f_xx shape: (86160,), mu shape: (), f shape: (86160,), K_secretion shape: (), theta_endocyto

ValueError: Exception encountered when calling layer "pinn_7" "                 f"(type PINN).

Input 0 of layer "dense_32" is incompatible with the layer: expected axis -1 of input shape to have value 4, but received input with shape (100, 3)

Call arguments received by layer "pinn_7" "                 f"(type PINN):
  • x=tf.Tensor(shape=(100, 3), dtype=float64)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import scienceplots
import latex
plt.style.use(['science','grid'])

In [2]:
# this will be my main diffusion equation
def diffusion_equation(x, t, D):
    return 1 / (4 * np.pi * D * t) * np.exp(-np.linalg.norm(x) ** 2 / (4 * D * t))

# Direlcht's and Neumanns boundary conditions depending we'll seeeeee
x = np.linspace(-1, 1, 50)
t = np.linspace(0.01, 1, 20)
X, T = np.meshgrid(x, t)
X_flat = X.flatten()
T_flat = T.flatten()
D_values = [0.1, 0.2, 0.3, 0.4]
N = X_flat.shape[0]

# Generating training data
X_train = np.zeros((N * len(D_values), 2))
y_train = np.zeros((N * len(D_values), 1))
for i, D in enumerate(D_values):
    y = diffusion_equation(np.vstack((X_flat, T_flat)).T, 1, D)
    X_train[i * N:(i + 1) * N, :] = np.vstack((X_flat, T_flat)).T
    y_train[i * N:(i + 1) * N, :] = y.reshape(-1, 1)

# randmoizw the training data and then i will choose the model
perm = np.random.permutation(N * len(D_values))
X_train = X_train[perm]
y_train = y_train[perm]


model = Sequential()
model.add(Dense(50, input_shape=(2,), activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))
model.summary()


optimizer = RMSprop(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)

# Training the model
history = model.fit(X_train, y_train, epochs=200, batch_size=256, verbose=1, validation_split=0.2)

#Plotting the training loss
plt.figure(figsize=(10,6))
plt.figure(figsize=(3.54,3.54), dpi=600)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
# plt.savefig('figure.png', bbox_inches='tight') just so i can save and  plot the graph
plt.show()


# setting up my predictions here
y_pred = []
for D in D_values:
    y = model.predict(np.vstack((X_flat, T_flat)).T)
    y_pred.append(y.reshape(X.shape))


# Plotting all coeficeints results
fig, axs = plt.subplots(2, 2, figsize=(8, 8), dpi=600)
for i in range(len(D_values)):
    ax = axs.flat[i]
    ax.contourf(X, T, y_pred[i])
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_title('D = {}'.format(D_values[i]))
fig = plt.savefig('all.png', bbox_inches='tight')
plt.show()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                150       
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,751
Trainable params: 2,751
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200


C:\Users\Ioannis\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


13/13 [==============================] - 1s 14ms/step - loss: 0.0224 - val_loss: 4.3051e-04
Epoch 2/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0112 - val_loss: 2.8735e-04
Epoch 3/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0096 - val_loss: 1.4356e-04
Epoch 4/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0073 - val_loss: 0.0011
Epoch 5/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0065 - val_loss: 0.0030
Epoch 6/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0048 - val_loss: 7.2071e-04
Epoch 7/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0039 - val_loss: 4.5419e-04
Epoch 8/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0033 - val_loss: 0.0020
Epoch 9/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0026 - val_loss: 6.3867e-04
Epoch 10/200
13/13 [==============================] - 0s 4ms/step - loss: 0.0024 - val_loss: 5.4

<Figure size 1000x600 with 0 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2124x2124 with 1 Axes>

32/32 [==============================] - 0s 710us/step


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 4800x4800 with 4 Axes>